In [1]:
import os
from tqdm import tqdm
import numpy as np
import cv2
import pandas as pd 
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity

2023-11-13 23:31:20.759677: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 23:31:20.793820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 23:31:20.793849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 23:31:20.793876: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 23:31:20.801175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 23:31:20.802620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

2023-11-13 23:31:23.214283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 23:31:23.258696: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def load_features(feature_folder):
    feature_vectors = []
    image_paths = []
    
    for filename in tqdm(os.listdir(feature_folder)):    
        feature_path = os.path.join(feature_folder, filename)
        feature_vector = np.load(feature_path)
        feature_vectors.append(feature_vector)
        image_paths.append(feature_path.replace(".npy", "").split("/")[-1])

    return np.array(feature_vectors), image_paths

In [4]:
def find_most_similar_images(query_features, all_features, image_paths, top_k=5):
    similarities = cosine_similarity(query_features.reshape(1, -1), all_features)
    similar_indices = similarities.argsort()[0][-top_k:][::-1]
    similar_images = [image_paths[i] for i in similar_indices]
    return similar_images

In [5]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [6]:
def extract_features_from_single_image(query_image_path):
    img_array = preprocess_image(query_image_path)
    features = base_model.predict(img_array)
    features = features.flatten()
    return features

In [17]:
def capture_image():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return
    cap.release()
    img = cv2.resize(frame, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    features = features.flatten()
    return features

In [18]:
df = pd.read_csv('./labels.csv')

In [19]:
feature_folder = './features2'
all_features, image_paths = load_features(feature_folder)

100%|██████████| 1124/1124 [00:01<00:00, 886.95it/s]


In [30]:
def get_curr_check(img):
    return df.loc[ df['file'].isin( img for img in find_most_similar_images(img, all_features, image_paths, top_k=10)), 'labels'].value_counts().sort_values(ascending=False).index[0]

In [32]:
curr_checkpoint = get_curr_check(capture_image())
prev_checkpoint = get_curr_check(capture_image())

1/1 [==============================] - 0s 58ms/step
